In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import *


В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [3]:
NUMBER_OF_HUMAN = 50
HEADHUNT_NUMBER_1 = 34
HEADHUNT_NUMBER_0 = 16
HEAD_1_SUCCESS = 10
HEAD_0_SUCCESS = 4

In [4]:
#   Выборки не связаны
mean_group_1 = HEAD_1_SUCCESS / HEADHUNT_NUMBER_1
mean_group_2 = HEAD_0_SUCCESS / HEADHUNT_NUMBER_0
print(f'mean 1: {mean_group_1}, mean 2: {mean_group_2}')

mean 1: 0.29411764705882354, mean 2: 0.25


In [10]:
def quantile_interval(data, alpha=0.05):
    z_quantile = stats.norm.ppf(1 - alpha/2.)
    p = sum(data) / len(data)
    add_sum = z_quantile * np.sqrt(p * (1 - p)/ len(data))
    return p - add_sum, p + add_sum

array1 = [1] * HEAD_1_SUCCESS + [0] * (HEADHUNT_NUMBER_1 - HEAD_1_SUCCESS)
array2 = [1] * HEAD_0_SUCCESS + [0] * (HEADHUNT_NUMBER_0 - HEAD_0_SUCCESS)
quantile_interval(array1)

(0.1409611811913692, 0.4472741129262779)

In [9]:
#   Чтобы не писать метод для Уилсона, он уесть в proportion_confint
print('YES:',proportion_confint(HEAD_1_SUCCESS, HEADHUNT_NUMBER_1, alpha=0.05, method='wilson'))
print('NO:',proportion_confint(HEAD_0_SUCCESS, HEADHUNT_NUMBER_0, alpha=0.05, method='wilson'))

YES: (0.16834630670422424, 0.4616890979471444)
NO: (0.10182067491213043, 0.4949831653550878)


###   Хочется понять значимость, а интервалы пересекаются, значит двухвыборочная несвязанная модель

In [19]:
def confint_not_linked_greater(data1, data2, alpha=0.05):
    p1 = sum(data1) / len(data1)
    p2 = sum(data2) / len(data2)
    P = (p1 * len(data1) + p2 * len(data2)) / (len(data1) + len(data2))
    Z_stat = (p1 - p2) / np.sqrt(P * (1 - P) * ((1 / len(data1)) + (1 / len(data2))))
    return stats.norm.sf(Z_stat)

print(f"{confint_not_linked_greater(array1, array2, alpha=.05):.4f}")

0.3729


### Задача о банкнотах

In [20]:
data = pd.read_csv('banknotes.txt', header=0, sep='\t')
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1
...,...,...,...,...,...,...,...
195,215.0,130.4,130.3,9.9,12.1,139.6,0
196,215.1,130.3,129.9,10.3,11.5,139.7,0
197,214.8,130.3,130.4,10.6,11.1,140.0,0
198,214.7,130.7,130.8,11.2,11.2,139.4,0


In [41]:
from sklearn import model_selection, linear_model, metrics

In [28]:
train, test = model_selection.train_test_split(data, test_size=50, random_state=1)
print("TEST SHAPE:", test.shape)

TEST SHAPE: (50, 7)


In [34]:
X_train = train.drop('real', axis=1)
y_train = train['real']
X_test = test.drop('real', axis=1)
y_test = test['real']

In [39]:
logistic_1_3 = linear_model.LogisticRegression(solver='liblinear')
logistic_4_6 = linear_model.LogisticRegression(solver='liblinear')

In [50]:
logistic_1_3.fit(X_train[['X1', 'X2', 'X3']], y_train)
logistic_4_6.fit(X_train[['X4', 'X5', 'X6']], y_train)
ans1 = logistic_1_3.predict(X_test[['X1', 'X2', 'X3']])
ans2 = logistic_4_6.predict(X_test[['X4', 'X5', 'X6']])

In [51]:
print('1-3',metrics.accuracy_score(y_test, ans1))
print('4-6', metrics.accuracy_score(y_test, ans2))

1-3 0.8
4-6 0.98


In [ ]:
#   Судя по всему выборки связанные так как мы заранее отделили 50 банкнот для теста

In [64]:
samp1 = [1 if y_test.values[i] == ans1[i] else 0 for i in range(len(ans1))]
samp2 = [1 if y_test.values[i] == ans2[i] else 0 for i in range(len(ans2))]
def z_criteria_linked(sample1, sample2, alpha=0.05):
    zipped = list(zip(sample1, sample2))
    n = len(zipped)
    f = sum([1 if x[0] == 0 and x[1] == 1 else 0 for x in zipped])
    g = sum([1 if x[0] == 1 and x[1] == 0 else 0 for x in zipped])
    Z_stat = (f-g) / (np.sqrt(f + g - (((f - g) ** 2) / n)))
    return 2 * stats.norm.sf(Z_stat)

def interval_linked(sample1, sample2, alpha=0.05):
    zipped = list(zip(sample1, sample2))
    n = len(zipped)
    f = sum([1 if x[0] == 0 and x[1] == 1 else 0 for x in zipped])
    g = sum([1 if x[0] == 1 and x[1] == 0 else 0 for x in zipped])
    z_quantile = stats.norm.ppf(1 - alpha/2.)
    add_sub = z_quantile * np.sqrt(((f + g)/ (n ** 2)) - ((f - g) ** 2/ (n ** 3)))
    return ((f - g) / n) - add_sub, ((f - g) / n) + add_sub
print("{0:e}".format(z_criteria_linked(samp1, samp2)))
print("[{0[0]:.4f}, {0[1]:.4f}]".format(interval_linked(samp2, samp1)))

3.296938e-03
[-0.3001, -0.0599]


Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [70]:
standart_mean = 525
standart_std = 100
n_group = 100
group_mean = 541.5
#   H0 - программа неэффективна
#   H1 group_mean > standart_mean
Statistic = (group_mean - standart_mean) / (standart_std / 10)
print(f"{stats.norm.sf(Statistic):.4f}")

0.0495
